In [1]:
import bs4 
import requests
import pandas as pd
import numpy as np
import os
import requests
import re
import datetime


In [2]:
now = datetime.datetime.now()
Today_date = now.strftime("%Y-%m-%d")

In [3]:
url = 'https://www.powertransmission.com/'
res = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
soup = bs4.BeautifulSoup(res.text,'lxml')

In [4]:
def complete_url(url):
    return ' http://www.powertransmission.com'+url

In [5]:
List = soup.find('ul',{'class':'menu'}).find('ul',{'class':'sub-menu'}).findAll('li')

In [6]:
records = []
for item in List[:4]:
    category = item.find('a').text
    url = item.find('a')['href']
    res = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
    soup = bs4.BeautifulSoup(res.text,'lxml') 
    sub_list = soup.find('div',{'class':'panel'}).findAll('p')

    for i in range(2,len(sub_list[1:]),2):
        try:
            Source = 'Power Transmissiom'
            Category = category
            Heading = sub_list[i].find('a').text
            URL = complete_url(sub_list[i].find('a')['href'])
            Text = '-'
            records.append((Source,Heading,Category,URL,Text))
        except:
            i=i+1
            continue

In [7]:
len(records)

508

In [8]:
df1 = pd.DataFrame(records,columns=['Source','Heading','Category','URL','Text'])

In [9]:
df1['Category'].value_counts()

Design Engineering    235
Gear Drives           106
Couplings             100
Bearings               67
Name: Category, dtype: int64

In [10]:
record2 = []
for item in List[4:]:
    category = item.find('a').text
    url = item.find('a')['href']
    res = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
    soup = bs4.BeautifulSoup(res.text,'lxml') 
    sub_list = soup.find('div',{'class':'panel'}).findAll('p')

    for i in range(3,len(sub_list[1:]),2):
        try:
            Source = 'Power Transmissiom'
            Category = category
            Heading = sub_list[i].find('a').text
            URL = complete_url(sub_list[i].find('a')['href'])
            Text = '-'
            record2.append((Source,Heading,Category,URL,Text))
        except:
            i=i+1
            continue

In [11]:
df2 = pd.DataFrame(record2,columns=['Source','Heading','Category','URL','Text'])

In [12]:
df2['Category'].value_counts()

Motors            245
Motion Control    160
Gears             130
Software           56
Name: Category, dtype: int64

In [13]:
df = pd.concat([df1[1:],df2[1:]])

In [14]:
df.reset_index(drop=True,inplace=True)

In [15]:
def get_text(url):

    try:
        res = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
        soup = bs4.BeautifulSoup(res.text,'lxml')
        txt_raw =soup.find('div',{'class':'article-head'}).findAll('p')
        i=0
        length=0
        for para in txt_raw:
            if (length < len(para.text)):
                maxi = i
                length = len(para.text)
            i=i+1
        text = txt_raw[maxi].text
    except:
        try:
            res = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
            soup = bs4.BeautifulSoup(res.text,'lxml')
        #    res = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
         #   soup = bs4.BeautifulSoup(res.text,'lxml')
            txt_raw =soup.find('div',{'class':'panel'}).findAll('p')
            gg = str()
            for para in txt_raw:
                gg = gg +' ' +str(para.text)
            gg = gg.replace('\xa0',' ')
            text = gg
        except:
            text = '-'
    return text

In [16]:
df['Text'] = df['URL'].apply(get_text)

In [23]:
blank = df[df['Text']=='  ' ]
df = df[df['Text']!='  ']

In [33]:
def fix_URL(URL):
    URL = URL[1:-1]
    
    return URL
blank['URL'] = blank['URL'].apply(fix_URL)

In [60]:
def get_text_blank(url):
    
        res = requests.get(url,headers={'User-Agent': 'Mozilla/5.0'})
        soup = bs4.BeautifulSoup(res.text,'lxml')
        text_list = soup.find('div',{'class':'panel'}).findAll('div',{'class':'photo'})
        text = ''
        for txt in text_list:
            text= text+txt.text
        text = text.replace('\n','')
        text = text.split('—')[1]
        text = text.split('URL')[0]
        return text

In [61]:
blank['Text'] = blank['URL'].apply(get_text_blank)

In [62]:
blank.head()

,Source,Heading,Category,URL,Text
1,Power Transmissiom,4 SKF Offers Sour Gas Compressor Bearings,Bearings,http://www.powertransmission.com/news/4999/SKF...,SKF recently announced the inclusion of SKF so...
2,Power Transmissiom,6 Bearings and Balls for Turbine Flow Meters,Bearings,http://www.powertransmission.com/news/2812/Bea...,The Boca Bearing Company recently announced a ...
10,Power Transmissiom,22 SKF Introduces Carb Roller Bearings,Bearings,http://www.powertransmission.com/news/5103/SKF...,SKF Carb toroidal roller bearings introduce ro...
47,Power Transmissiom,96 Pacamor Kubar Bearings Purchased by Two Com...,Bearings,http://www.powertransmission.com/news/6686/Pac...,Pacamor Kubar Bearings (PKB) was recently been...
50,Power Transmissiom,102 KISSsoft and SKF Collaborate on Rolling Be...,Bearings,http://www.powertransmission.com/news/7290/KIS...,The new version of KISSsoft 03/2016 now contai...


In [65]:
df = df[df['Text']!='  ']
df = pd.concat([df,blank])


In [67]:
df.reset_index(drop=True,inplace=True)

In [68]:
df.to_csv('Power Transmission.csv',index=False)